In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("ACC_5min.csv")
df.tail()

,Unnamed: 0,Date,Time,Interval,Open,High,Low,Close
60940,60940,2021-07-28,15:05,2021-07-28 15:05,2391.75,2393.90,2391.75,2393.15
60941,60941,2021-07-28,15:10,2021-07-28 15:10,2392.20,2393.90,2392.15,2393.40
60942,60942,2021-07-28,15:15,2021-07-28 15:15,2393.40,2397.00,2393.15,2394.00
60943,60943,2021-07-28,15:20,2021-07-28 15:20,2393.90,2397.10,2393.10,2395.35
60944,60944,2021-07-28,15:25,2021-07-28 15:25,2395.35,2396.75,2384.80,2386.10


In [3]:
df.rename(columns = {"Interval" : "date"}, inplace = True)
df.head()
df.loc[:,["date","Open","High","Low","Close"]].head()

,date,Open,High,Low,Close
0,2018-01-01 09:15,1763.90,1763.90,1758.65,1758.65
1,2018-01-01 09:20,1758.65,1761.00,1754.60,1754.80
2,2018-01-01 09:25,1754.80,1756.35,1748.70,1750.15
3,2018-01-01 09:30,1750.15,1751.25,1748.00,1749.40
4,2018-01-01 09:35,1749.20,1751.15,1748.45,1749.25


In [4]:
imbalance_candle = "2018-01-18 12:10"
imbalance_candle = df[df["date"]== imbalance_candle].index[0]

In [5]:
min(df.loc[755:785]["Low"])

1840.0

In [6]:
def is_body_size_greater_than_3x_average(dataframe, index,atr_multiple,look_back):
    body_sizes = (dataframe['Close'] - dataframe['Open']).iloc[index - look_back:index].abs()
    average_body_size = body_sizes.mean()
    current_body_size = abs(dataframe['Close'].iloc[index] - dataframe['Open'].iloc[index])
    return current_body_size > atr_multiple * average_body_size

In [7]:
def imbalance_zone(imbalance_candle,df,look_back):
    
    
    verify_candle  = imbalance_candle+1
    
    
    for i in range(look_back):
        
        if is_body_size_greater_than_3x_average(df,imbalance_candle,3,look_back):
            upper = min(df["Low"][imbalance_candle-30:imbalance_candle])
            lower = df["High"][verify_candle] 

            if (upper-lower) > 0 :
                return {"Upper": upper, "Lower" : lower}  ## test in future the difference of lines more than some % of body

            else :
                return
        else:
            return

In [8]:
(imbalance_zone(1575,df,30))

{'Upper': 1675.1, 'Lower': 1635.0}

In [9]:
imbalance_candle = 1575
min(df["Low"][imbalance_candle-30:imbalance_candle])

1675.1

In [36]:
import plotly.graph_objs as go

df_plot = df[1500:1900]
candlestick = go.Candlestick(x=df_plot.index, open=df_plot['Open'], 
                             close=df_plot['Close'], high=df_plot['High'], low=df_plot['Low'])

fig = go.Figure(data=[candlestick])
fig.show()

In [31]:
imbalance_candle = [i for i in range(1576,1800)]

In [40]:
latest_three_levels = [] #fifo datastructure

for i in range(len(imbalance_candle)):
    levels = imbalance_zone(imbalance_candle[i],df,30)
    if levels:
        date = df["date"][imbalance_candle[i]]
        index = imbalance_candle[i]
        print(f"Date: {date} and Index: {index}")

        latest_three_levels.append(levels)
        if len(latest_three_levels) > 3:
            latest_three_levels.pop(0)

Date: 2018-02-06 09:15 and Index: 1650
Date: 2018-02-07 14:10 and Index: 1784
Date: 2018-02-07 15:15 and Index: 1797
Date: 2018-02-07 15:20 and Index: 1798


In [41]:
latest_three_levels

[{'Upper': 1597.0, 'Lower': 1595.8},
 {'Upper': 1590.85, 'Lower': 1588.35},
 {'Upper': 1587.0, 'Lower': 1581.95}]

In [ ]:
from collections import defaultdict
def to_intervals(data,n):
    # Read in the data from the text file
    data
    data = data.reset_index()
    data = np.array(data)
    """data = []
    name = "data_2017.txt" #input()
    with open(name, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            # Ignore the first column (stock name)
            data.append(row[1:])"""

    # Convert the data to n-minute intervals
    interval_data =  defaultdict(list)
    for row in data:
        date = row[1].split()[0]
        time = row[1].split()[1].split("+")[0][:-3]
        if time == "09:08:00":
            continue

        open_price = float(row[2])
        high_price = float(row[3])
        low_price = float(row[4])
        close_price = float(row[5])
        volume = float(row[6])
        # Extract the interval from the time and use it as the key in the dictionary
        interval = int(time[-2:]) // n
        interval = f"{time[:2]}:{interval * n:02d}"
        interval = str(date) +" " + interval
        interval_data[interval].append((date, time, open_price, high_price, low_price, close_price, volume))

    # Calculate the final open, high, low, and close prices for each interval
    final_prices = {}
    for interval, prices in interval_data.items():
        # Unpack the tuples into separate lists for each price type
        dates, times, opens, highs, lows, closes, volume = zip(*prices)
        # The final open price is the same as the first open price in the interval
        final_open = opens[0]

        # The final high price is the highest of all the high prices in the interval
        final_high = max(highs)
        # The final low price is the lowest of all the low prices in the interval
        final_low = min(lows)
        # The final close price is the same as the last close price in the interval
        final_close = closes[-1]
        # The final volume will be the sum of all volumes of the candels
        final_volume = sum(volume)
        # Store the final prices in a tuple in the dictionary
        final_prices[interval] = (dates[0], times[0], final_open, final_high, final_low, final_close, final_volume)
    
    ws = []
    # Write the data rows
    for interval, final in final_prices.items():
        date, time, open_price, high_price, low_price, close_price, volume = final
        ws.append([date, time, interval, open_price, high_price, low_price, close_price, volume])
    return pd.DataFrame(ws,columns = ["Date", "Time", "Interval", "open", "high", "low", "close","volume"])
    

def handle_data(context,data):
    ### check on index n-1::
    ## Fetch n-1 candles volume candles:::
    data = data.reset_index()
    
    ## exit prev conditions
    if context["position"] = True:
        if data["close"]>sum(context["priceLevels"])/2:
            context["profit"] += data["close"].iloc[-1] - context["buy"]
            context["profitList"].append(context["profit"])
            context["position"]  = 0
            context["trades"] += 1
        elif data["close"]<context["stoploss"]:
            context["losses"] += 1
            context["position"] = 0
            context["profit"] += data["close"].iloc[-1] - context["buy"]
            context["profitList"].append(context["profit"])
            context["trades"] += 1
        return
        
    #volume_candles = f("time of n-1 candle")
    """data["signal"] = 0
    for i in range(-len(data),-1,-1):
        data["signal"][i] = volCheck(data.iloc[-i-500:-i],data.iloc[i].index[0]-15,data.iloc[i].index[0])
        
    return data"""
    
    ## apply volume threshold
    #curr_vol>200
    
    
    candleToCheck = data["datetime"].iloc[-1]
    
    ## apply volume candle conditions
    if not imbalance_from_volume(data,candleToCheck):
        return
    
    ## apply price conditions 
    ## between previous 30 5 minute candles low, current candle high -- range of body of imbalance greater than threshold
    ## 5 min data
    data_5min = to_intervals(data.iloc[-5000:],5)
    
    p1,p2 = isPriceCond(data_5min) ##zone funciton....
    
    if p1==p2:
        return
    
    #context["priceLevels"] = [p1,p2]

    
    context["priceLevels"] = [p1,p2]
    
    strategy_trade(context,data_5min)
    
    

def buy(context,price):
    context["position"] = True
    context["buy"] = price
    
def strategy_trade(context,df):
    data = df.iloc[-50:]
    data["mid"] = (data["high"]+data["low"])/2
    data["ao"] = data["mid"].rolling(window = 5).mean() - data["mid"].rolling(window = 34).mean()
    signals = data[(data["ao"].shift(4)< data["ao"].shift(3)) & (data["ao"].shift(3)< data["ao"].shift(2)) & (data["ao"].shift(2) > data["ao"])]
    if len(signal):
        ind = signal.index[-1]
        datetime = signal["datetime"][ind]
        buy(context,data["open"].iloc[-1])
        
    #data[data["ao"].shift(3)< data["ao"].shift(2) and data["ao"].shift(2)< data["ao"].shift(1) and data["ao"].shift(1) > data["ao"]]